In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import re
import cv2
import numpy as np
from torchvision import transforms

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 32 * 32, 256)  # Adjusted input size
        self.fc2 = nn.Linear(256, 2) # predicting two variables (alfa x and betax)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class CustomDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = sorted(os.listdir(image_dir))
        self.pattern = r"epsnx([\d.-]+)_alfax([\d.-]+)_betax([\d.-]+)_epsny([\d.-]+)_alfay([\d.-]+)_betay([\d.-]+)_epsnz([\d.-]+)_alfaz([\d.-]+)_betaz([\d.-]+)\.png"
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_name = self.image_files[index]
        image_path = os.path.join(self.image_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transform(image)
        matches = re.search(self.pattern, image_name)
        variables = [float(matches.group(i)) for i in range(1, 10) if matches.group(i)]
        alfa_x = variables[1] # alfax value
        beta_x = variables[2] # betax value
        return image, torch.tensor([alfa_x, beta_x]) # returns alfa x and betax as the labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 32
learning_rate = 0.001
num_epochs = 10

model = CNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

image_dir = "images"
dataset = CustomDataset(image_dir)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

total_steps = len(dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.4f}")

torch.save(model.state_dict(), "cnn_model.pth")


In [8]:
import torchvision.transforms as transforms
import cv2

model = CNN()
model.load_state_dict(torch.load("cnn_model.pth"))
model.eval()

# using cv2 is gross but i like the transforms more
image_path = "test\epsnx0.10_alfax2.15_betax54.53_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00.png"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (64, 64))  # Adjusted size
image = transforms.ToTensor()(image)
image = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image)
image = image.unsqueeze(0)

with torch.no_grad():
    output = model(image)
    predicted_variables = output.squeeze().tolist()

print("Predicted variables:", predicted_variables)

Predicted variables: [0.0771125927567482, 0.03757302463054657]
